In [1]:
import os
import numpy as np
import pandas as pd
import glob
import re
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sktime.datasets import load_from_tsfile_to_dataframe
import warnings

warnings.filterwarnings('ignore')

c:\Users\92469\miniconda3\envs\thuml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Dataset_Custom_WithouDate(Dataset):
    def __init__(self, root_path, flag='train', size=None,
                 features='S', data_path='ETTh1.csv',
                 target='OT', scale=True, timeenc=0, freq='h', seasonal_patterns=None):
        # size [seq_len, label_len, pred_len]
        # info
        if size == None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]
        # init
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]

        self.features = features
        self.target = target
        self.scale = scale

        self.root_path = root_path
        self.data_path = data_path
        self.__read_data__()

    def __read_data__(self):
        self.scaler = StandardScaler()
        df_raw = pd.read_csv(os.path.join(self.root_path,
                                          self.data_path))

        '''
        df_raw.columns: [...(other features), target feature]
        '''
        cols = list(df_raw.columns)
        cols.remove(self.target)
        df_raw = df_raw[cols + [self.target]]
        num_train = int(len(df_raw) * 0.7)
        num_test = int(len(df_raw) * 0.2)
        num_vali = len(df_raw) - num_train - num_test
        border1s = [0, num_train - self.seq_len, len(df_raw) - num_test - self.seq_len]
        border2s = [num_train, num_train + num_vali, len(df_raw)]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]

        df_data = df_raw
        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]]
            self.scaler.fit(train_data.values)
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values
        self.mean = data.mean()

        self.data_x = data[border1:border2]
        self.data_y = data[border1:border2]

       
    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        return seq_x, seq_y

    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)


In [3]:
from torch.utils.data import DataLoader
data_set = Dataset_Custom_WithouDate(
    root_path="../../dataset/LongYuanPower/processed",
    flag='test',
    size=[288,144,144],
    features='MS',
    target='Wspd',
    data_path='Turb1.csv',
    scale=True
)
mean = data_set.mean
data_loader = DataLoader(
    data_set,
    batch_size=128,
    drop_last=True,
    num_workers=0
)

In [4]:
# import pandas as pd
# import numpy as np
# df = pd.read_csv('../../dataset/LongYuanPower/processed/Turb1.csv')['Wspd'].to_numpy()
# mean = df.mean()
print(mean)

-0.060166859257820504


In [5]:
import numpy as np

def MAE(pred, true):
    return np.mean(np.abs(pred - true))


def MSE(pred, true):
    return np.mean((pred - true) ** 2)


def metric(pred, true):
    mae = MAE(pred, true)
    mse = MSE(pred, true)

    return mae, mse


In [6]:
trues = []
preds= []
print(data_loader.__len__())
for i, (batch_x, batch_y) in enumerate(data_loader):
    true = batch_y.float().numpy()
    trues.append(true) 
trues = np.array(trues,dtype=np.float64)
preds = np.ones_like(trues,dtype=np.float64) * mean
preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])
print(preds[0])
print(trues[0])
mae, mse = metric(preds, trues)
print('mse:{}, mae:{}'.format(mse, mae))

54
[[-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686]
 [-0.06016686 -0.06016686

#### 平均值 
mse:0.9701624722239056, mae:0.8466369882041933

#### DLinear  
mse:0.9231436848640442, mae:0.798798143863678

#### Autoformer 
mse:1.2911731004714966, mae:0.8909386992454529

#### Informer 
mse:0.919448971748352, mae:0.7650326490402222

#### LSTM
mse:1.0110634565353394, mae:0.8453809022903442